# goal: automate customer support using AI agents

## notes and questions
- the `allow_delegation` attribute for agents is True by default. we need to set it to False explicitly when we want agents to do their job without delegating to any other agent or agents.
- setting `allow_delegation` to True does not mean that the agent will 100% delegate work. since the process is fuzzy and non-deterministic, the agent may or may not make the call to delegate. in the case below, the QA agent with delegation enabled may accept the first response the support rep agent gives it or it may decide that the support rep agent needs to improve its output and delegate the task of coming up with an improved response back to it.
- agents can be provided tools on two levels:
  1. agent-level: the agent can use the tool for any task it performs
  2. task-level: the agent can use the tool only when performing that specific task; task tools override agent tools
- we can enable memory by setting `memory` to True in the `Agent()` function as well as the `Crew()` function.
  -  q. how does the *place (function)* in which memory is enabled affect how memory is retained? does enabling it for an agent mean that only memory pertaining to that agent will be retained and enabling it for the crew mean that memory pertaining to all the agents (and tasks) within that crew will be retained?

## main code

In [1]:
import warnings
warnings.filterwarnings('ignore')  # don't bother showing warnings

from crewai import Agent, Task, Crew

from IPython.display import Markdown

import os
from utils import get_openai_api_key
openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'

In [2]:
# agents

# support rep
support_agent = Agent(
    role="Senior Support Representative",  # senior implies that this agent should be better than a newbie support rep
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,  # the default is True
	verbose=True
)


# quality assurance specialist
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    allow_delegation=True,  # this is True by default, but I am setting it explicitly to emphasize that this guy can delegate work back to the support rep agent
	verbose=True
)

In [8]:
# tools

from crewai_tools import ScrapeWebsiteTool, WebsiteSearchTool

scrape_website_tool = ScrapeWebsiteTool(website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/")

In [4]:
# tasks

# resolving asked questions/queries
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[scrape_website_tool],  # we give access to the scrape_website_tool on task-level
    agent=support_agent,
)


# reviewing provided resolution
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        "find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [5]:
# assembling the crew

crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],  # the order matters here
  verbose=2,
  memory=True  # enables memory on crew level
)

In [6]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide the best support to DeepLearningAI by thoroughly addressing Andrew Ng's request for guidance on adding memory to a Crew in CrewAI. I should use the tool "Read website content" to gather information on how to set up a Crew and add memory to it.

Action: 
Read website content

Action Input: 
{
  "url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
}
 

Introduction - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIc

In [7]:
Markdown(result)

Dear Andrew Ng,

Thank you for reaching out with your inquiry on how to add memory to a Crew in CrewAI. I understand the importance of this task for DeepLearningAI, and I am here to provide you with a comprehensive guide on setting up a Crew and adding memory to it.

Memory management plays a crucial role in optimizing the performance and efficiency of deep learning tasks. By implementing various memory management strategies, you can enhance memory utilization, prevent memory leaks, and ensure smooth operation of your AI workflows.

To add memory to a Crew in CrewAI, you can follow these key steps:

1. Create a Crew:
   - Start by creating a Crew in CrewAI, which serves as the central organization overseeing AI agent teams and workflows.

2. Add AI Agents:
   - Within the Crew, add specialized AI agents with distinct roles such as researchers, writers, or analysts.

3. Implement Memory Management Strategies:
   - Utilize memory pooling, caching, and garbage collection techniques to efficiently manage memory within the Crew.

4. Equip Agents with Memory Tools:
   - Provide AI agents with custom tools and APIs that enable them to interact with memory resources and optimize memory usage.

5. Monitor and Optimize Memory Usage:
   - Continuously monitor memory usage within the Crew and optimize memory allocation to ensure efficient operation.

By incorporating these steps, you can effectively add memory to your Crew in CrewAI and enhance the performance of your AI agent teams.

For more technical details on memory management strategies and tools within CrewAI, you can refer to the documentation available at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/. This resource offers in-depth information on creating Crews, managing AI agents, and optimizing memory utilization.

In practical scenarios, adding memory to a Crew in CrewAI offers benefits such as improved task performance, faster data processing, and enhanced overall efficiency in AI-driven workflows. By implementing memory management strategies and equipping AI agents with memory tools, you can optimize resource utilization and achieve better outcomes in your AI projects.

If you have any further questions or need additional assistance with adding memory to your Crew in CrewAI, please feel free to reach out. I am here to help you every step of the way.

Best regards,
[Your Name]
Senior Support Representative
CrewAI